In [ ]:
!pip install -q wandb py_vncorenlp

import py_vncorenlp
py_vncorenlp.download_model(save_dir='/kaggle/working/')
import os
os.environ['JAVA_HOME'] = '/usr/lib/jvm/default-java'
rdrsegmenter = py_vncorenlp.VnCoreNLP(annotators=["wseg"], save_dir='/kaggle/working')

In [1]:
!pip install jsonlines
import jsonlines

In [2]:
def read_jsonl(file_path):
    data = []
    with jsonlines.open(file_path, mode='r') as reader:
        for item in reader:
            data.append(item)
    return data

all_laws = read_jsonl('/kaggle/input/law-qa/laws_merged.jsonl')

In [3]:
import unicodedata

def normalize_text(text):
    text = text.lower()
    text = unicodedata.normalize('NFD', text)
    text = ''.join([c for c in text if not unicodedata.combining(c)])
    
    return text

In [4]:
import re

def extract_article_id(law_identifier, article_name):
    law_identifier_lower = law_identifier.lower()
    article_name_normalized = normalize_text(article_name)

    match = re.search(r'đieu (\d+)', article_name_normalized)
    
    if match:
        article_number = match.group(1)
        article_id = f"{law_identifier_lower}_{article_number}"
    else:
        match = re.search(r'ðieu (\d+)', article_name_normalized)
        if match:
            article_number = match.group(1)
            article_id = f"{law_identifier_lower}_{article_number}"
        else: article_id = None

    return article_id

In [5]:
no_segmented_corpus = []
def flatten_law(law_element, flattened_articles, law_identifier):
    if 'parts' in law_element:
        for part in law_element['parts']:
            flatten_law(part, flattened_articles, law_identifier)
    if 'chapters' in law_element:
        for chapter in law_element['chapters']:
            flatten_law(chapter, flattened_articles, law_identifier)
    if 'articles' in law_element:
        for article in law_element['articles']:
            if 'name' in article and 'content' in article:
                no_segmented_corpus.append({
                    'article_id': extract_article_id(law_identifier, article['name']),
                    'name': article['name'],
                    'content': article['content']
                })

In [6]:
for law in all_laws:
    flatten_law(law, no_segmented_corpus, law['identifier'])

In [7]:
no_segmented_corpus = [item for item in no_segmented_corpus if item.get('name', '').strip() != '']

In [8]:
no_segmented_corpus[100]

{'article_id': '32/2024/qh15_101',
 'name': 'Điều 101. Quy định nội bộ',
 'content': ['1. Căn cứ vào quy định của Luật này, quy định của Thống đốc Ngân hàng Nhà nước và quy định khác của pháp luật có liên quan, tổ chức tín dụng phải xây dựng và ban hành quy định nội bộ đối với hoạt động nghiệp vụ của tổ chức tín dụng, bao gồm cả việc thực hiện hoạt động nghiệp vụ bằng phương tiện điện tử, bảo đảm có cơ chế kiểm soát, kiểm toán nội bộ, quản lý rủi ro gắn với từng quy trình nghiệp vụ kinh doanh, phương án xử lý trường hợp khẩn cấp.',
  '2. Tổ chức tín dụng phải ban hành quy định nội bộ về các nội dung sau đây:',
  'a) Cấp tín dụng, quản lý khoản cấp tín dụng;',
  'b) Phân loại tài sản có, trích lập và sử dụng dự phòng rủi ro;',
  'c) Đánh giá chất lượng tài sản có và tuân thủ tỷ lệ an toàn vốn tối thiểu;',
  'd) Quản lý thanh khoản, trong đó có thủ tục và giới hạn quản lý thanh khoản;',
  'đ) Kiểm soát nội bộ và kiểm toán nội bộ phù hợp với tính chất và quy mô hoạt động của tổ chức tín d

In [10]:
with jsonlines.open("/kaggle/working/flatten_law_corpus.json", mode='w') as writer:
    for item in no_segmented_corpus:
        writer.write(item)

In [10]:
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained('bkai-foundation-models/vietnamese-bi-encoder')

tokenizer_config.json:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/22.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

In [11]:
# !pip install -qU langchain-text-splitters
# from langchain_text_splitters import RecursiveCharacterTextSplitter,SentenceTransformersTokenTextSplitter

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [13]:
no_segmented_queries = read_jsonl('/kaggle/input/law-qa/query_set_evaluate.json')

In [14]:
no_segmented_queries[10]

{'query_id': '66635835c2f544363ef20db0',
 'query': '03 độ mật của bí mật nhà nước là gì?',
 'relevant_docs': ['29/2018/qh14_8']}

In [15]:
segmented_queries = []

for query in no_segmented_queries:
    segmented_query = ''.join(rdrsegmenter.word_segment(query['query']))
    relevant_docs = []
    for doc in query['relevant_docs']:
        relevant_docs.append(doc)
    segmented_queries.append({'query_id':query['query_id'],
                              'query' : segmented_query,
                              'relevant_docs' : relevant_docs
                            })

In [16]:
segmented_queries[10]

{'query_id': '66635835c2f544363ef20db0',
 'query': '03 độ mật của bí_mật nhà_nước là gì ?',
 'relevant_docs': ['29/2018/qh14_8']}

In [17]:
for corpus in no_segmented_corpus:
    corpus['text'] = '\n'.join([str(content_part) for content_part in corpus['content']])
    del corpus['content']  

In [18]:
def chunk_text_with_subheadings(text):
    if re.search(r'\d+\.', text) or re.search(r'[a-z]\)', text, flags=re.IGNORECASE):
        chunks = re.split(r'\n(?=\d+\.)', text)
        final_chunks = [chunk.strip() for chunk in chunks]
    else:
        final_chunks = [text.strip()]
    
    return final_chunks

In [19]:
test_chunk = chunk_text_with_subheadings(no_segmented_corpus[100]['text'])
print(test_chunk)

['1. Căn cứ vào quy định của Luật này, quy định của Thống đốc Ngân hàng Nhà nước và quy định khác của pháp luật có liên quan, tổ chức tín dụng phải xây dựng và ban hành quy định nội bộ đối với hoạt động nghiệp vụ của tổ chức tín dụng, bao gồm cả việc thực hiện hoạt động nghiệp vụ bằng phương tiện điện tử, bảo đảm có cơ chế kiểm soát, kiểm toán nội bộ, quản lý rủi ro gắn với từng quy trình nghiệp vụ kinh doanh, phương án xử lý trường hợp khẩn cấp.', '2. Tổ chức tín dụng phải ban hành quy định nội bộ về các nội dung sau đây:\na) Cấp tín dụng, quản lý khoản cấp tín dụng;\nb) Phân loại tài sản có, trích lập và sử dụng dự phòng rủi ro;\nc) Đánh giá chất lượng tài sản có và tuân thủ tỷ lệ an toàn vốn tối thiểu;\nd) Quản lý thanh khoản, trong đó có thủ tục và giới hạn quản lý thanh khoản;\nđ) Kiểm soát nội bộ và kiểm toán nội bộ phù hợp với tính chất và quy mô hoạt động của tổ chức tín dụng;\ne) Hệ thống xếp hạng tín dụng nội bộ đối với tổ chức tín dụng phải xây dựng hệ thống xếp hạng tín dụn

In [20]:
 def count_tokens(text):
    tokens = tokenizer.tokenize(text)
    return len(tokens)

In [21]:
chunk_id_mapping = {}
chunk_no_segmented_corpus = []
max_chunk_size = 255
num_over_input = 0
longest_chunk = 0

for doc in no_segmented_corpus:
    list_chunk = chunk_text_with_subheadings(doc['text'])
    chunk_postfix = 1
    list_chunk_id = []
    
    for chunk in list_chunk:
        if len(chunk) > 5:
            text = ""

            text = doc['name'] + " : " + chunk
            chunk_id = doc['article_id'] + "_" + str(chunk_postfix)
            
            list_chunk_id.append(chunk_id)
            
            chunk_no_segmented_corpus.append({
            'doc_id' : doc['article_id'],
            'chunk_id' : chunk_id,
            'title' : doc['name'],
            'chunk' : chunk,
            'text' : text
            })
            
            chunk_postfix += 1
            
            if count_tokens(text) > longest_chunk:
                longest_chunk = count_tokens(text)
            
            if count_tokens(text) > max_chunk_size:
                num_over_input += 1
                
    chunk_id_mapping[doc['article_id']] = list_chunk_id

In [22]:
segmented_chunk_corpus = []

for corpus in chunk_no_segmented_corpus:
    segmented_chunk_corpus.append({
        'doc_id' : corpus['doc_id'],
        'chunk_id': corpus['chunk_id'],
        'title' : ''.join(rdrsegmenter.word_segment(corpus['title'])),
        'chunk' : ''.join(rdrsegmenter.word_segment(corpus['chunk'])),
        'text' : ''.join(rdrsegmenter.word_segment(corpus['text']))
    })

In [23]:
segmented_chunk_corpus[10000]

{'doc_id': '73/2021/qh14_26',
 'chunk_id': '73/2021/qh14_26_3',
 'title': 'Điều 26 .Lập danh_sách người sử_dụng trái_phép chất ma_tuý',
 'chunk': '3 . Công_an cấp xã giúp Uỷ_ban_nhân_dân cùng cấp đưa ra khỏi danh_sách người sử_dụng trái_phép chất ma_tuý trong các trường_hợp sau đây : a ) Người sử_dụng trái_phép chất ma_tuý không có hành_vi sử_dụng trái_phép chất ma_tuý trong thời_gian quản_lý quy_định tại khoản 2 Điều 23 của Luật này ; b ) Người sử_dụng trái_phép chất ma_tuý thuộc trường_hợp dừng quản_lý quy_định tại khoản 5 Điều 23 của Luật này ; c ) Người sử_dụng trái_phép chất ma_tuý chuyển đến cư_trú ở địa_phương khác .',
 'text': 'Điều 26 .Lập danh_sách người sử_dụng trái_phép chất ma_tuý :3 . Công_an cấp xã giúp Uỷ_ban_nhân_dân cùng cấp đưa ra khỏi danh_sách người sử_dụng trái_phép chất ma_tuý trong các trường_hợp sau đây : a ) Người sử_dụng trái_phép chất ma_tuý không có hành_vi sử_dụng trái_phép chất ma_tuý trong thời_gian quản_lý quy_định tại khoản 2 Điều 23 của Luật này ; b )

In [24]:
print(len(segmented_chunk_corpus))
print(num_over_input)
print(longest_chunk)
print(len(chunk_id_mapping))
print(len(no_segmented_corpus))

92351
3803
8140
28184
28314


In [25]:
def find_chunk(id):
    for chunk in segmented_chunk_corpus:
        if chunk['chunk_id'] == id:
            return chunk

In [26]:
def is_chunk_identifier(doc_id):
    return doc_id.count('_') == 2

In [27]:
segmented_queries_relevant_chunks = []

for query in segmented_queries:
    
    relevant_chunks = []
    
    for relevant_doc in query['relevant_docs']:
        if is_chunk_identifier(relevant_doc):
            relevant_chunks.append(relevant_doc)
        else:
            relevant_chunks += chunk_id_mapping.get(relevant_doc, [])
    
    segmented_queries_relevant_chunks.append({
        'query_id' : query['query_id'],
        'query' : query['query'],
        'relevant_docs' : query['relevant_docs'],
        'relevant_chunks' : relevant_chunks
    })

In [28]:
segmented_queries_relevant_chunks[15]

{'query_id': '66635804c2f544363ef12832',
 'query': '04 nguyên_tắc phòng cháy chữa_cháy ?',
 'relevant_docs': ['27/2001/qh10_4'],
 'relevant_chunks': ['27/2001/qh10_4_1',
  '27/2001/qh10_4_2',
  '27/2001/qh10_4_3',
  '27/2001/qh10_4_4']}

In [29]:
chunk_id_mapping['32/2024/qh15_135']

['32/2024/qh15_135_1',
 '32/2024/qh15_135_2',
 '32/2024/qh15_135_3',
 '32/2024/qh15_135_4',
 '32/2024/qh15_135_5']

In [30]:
import pickle
with open('/kaggle/working/chunk_corpus', 'wb') as f:
    pickle.dump(segmented_chunk_corpus, f)
    
with open('/kaggle/working/chunk_id_mapping', 'wb') as f:
    pickle.dump(chunk_id_mapping, f)
    
with open('/kaggle/working/queries_relevant_chunks', 'wb') as f:
    pickle.dump(segmented_queries_relevant_chunks, f)